In [1]:
import pandas as pd
import numpy as np

# Importando os dados já filtrados

In [2]:
cdr = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/cdr_after_filtes.csv')
cdr = cdr.drop('Unnamed: 0',1)
cdr.head()

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14,DAY_OF_WEEK
0,2013-03-21,21:03:30,0.13,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210,Thursday
1,2013-03-21,20:30:29,7.02,61,BA4DEA1F710C1216F74F1255150C49D6,61,4F6DCBA36292B8487743E13319E67DF6,51601,77,LOCAL,BRT,GVT,PRE PAGO,300000210,Thursday
2,2013-03-21,01:10:47,0.83,32,1AC8490BA99709F652B4A5C3525D3C82,32,B6BB8A38B286A41978E732BDE0249DB5,50141,117,LOCAL,OI,OI,POS PAGO,300000188,Thursday
3,2013-03-21,21:11:55,6.87,32,05AC98D91ED3C2821F53519B19CCBC58,32,A3ACD65D308B86F4487DB35A5F2E50EB,50141,117,LOCAL,OI,OI,PRE PAGO,300000533,Thursday
4,2013-03-21,20:29:21,0.13,61,BA4DEA1F710C1216F74F1255150C49D6,21,A49BA005920B8BA24BEE40F455B69B9B,51601,117,LOCAL,BRT,OI,PRE PAGO,300000210,Thursday


# Importando a localização das antenas

In [3]:
#É possível um mesmo conjunto de LAT e LONG terem mais de um Id atrelado
antennas = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/antenas_regiao_imediata_sjdr.csv', error_bad_lines = False, sep=';')
antennas = antennas.rename(columns={'id': 'ID', 'antenna': 'ANTENNA', 'city': 'CITY', 'lat': 'LAT', 'long':'LONG'})
antennas.head()

,ID,ANTENNA,CITY,LAT,LONG
0,4593,27712,Madre de Deus de Minas,-21.48194,-44.33039
1,4706,23442,São João del-Rei,-21.10707,-44.21979
2,4707,23463,São João del-Rei,-21.12699,-44.26144
3,4863,23462,São João del-Rei,-21.12699,-44.26144
4,5001,29943,Tiradentes,-21.11886,-44.16478


In [4]:
#O arquivo unique_locations contém as localizações únicas das antenas
#Atribui um identiifcador único pra elas, afim de adicionar esse identificador no dataframe de antenas
antennas_unique = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/unique_locations.txt', sep=']', header=None)
antennas_unique[0] = antennas_unique[0].str.replace('[','')
ids = ['MDM', 'SJDR1', 'SJDR2', 'SJDR3', 'SJDR4', 'SJDR5','TR1', 'TR2', 'LD1', 'RT', 'SVM', 'LD2', 'ST']
antennas_unique['ANTENNA_ID'] = ids
antennas_unique = antennas_unique.rename(columns={0:'ANTENNA', 1: 'CITY'})
antennas_unique.head()

,ANTENNA,CITY,ANTENNA_ID
0,"27711,27712,27713,48582,27451,48581",Madre de Deus de Minas,MDM
1,"23441,23442,50153,23443,50152,50151",São João del-Rei,SJDR1
2,"23461,23463,23462,50933,50931,50932",São João del-Rei,SJDR2
3,"23471,23472,23473,51311,51312",São João del-Rei,SJDR3
4,"23451,23452,23453,50663,50661,50662",São João del-Rei,SJDR4


In [5]:
antennas_with_id = antennas.merge(antennas_unique.assign(ANTENNA=antennas_unique['ANTENNA'].str.split(','))
               .explode('ANTENNA').assign(ANTENNA=lambda x: x['ANTENNA'].astype(int)), on='ANTENNA')

antennas_with_id = antennas_with_id.drop('CITY_y', 1)
antennas_with_id = antennas_with_id.rename(columns={'CITY_x': 'CITY'})
antennas_with_id.head()

,ID,ANTENNA,CITY,LAT,LONG,ANTENNA_ID
0,4593,27712,Madre de Deus de Minas,-21.48194,-44.33039,MDM
1,4706,23442,São João del-Rei,-21.10707,-44.21979,SJDR1
2,4707,23463,São João del-Rei,-21.12699,-44.26144,SJDR2
3,4863,23462,São João del-Rei,-21.12699,-44.26144,SJDR2
4,5001,29943,Tiradentes,-21.11886,-44.16478,TR1


In [6]:
cdr_merge_antennas = pd.merge(cdr, antennas_with_id[['ANTENNA', 'ANTENNA_ID']], on='ANTENNA', how='left')
cdr_merge_antennas.head(7)

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14,DAY_OF_WEEK,ANTENNA_ID
0,2013-03-21,21:03:30,0.13,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210,Thursday,RT
1,2013-03-21,20:30:29,7.02,61,BA4DEA1F710C1216F74F1255150C49D6,61,4F6DCBA36292B8487743E13319E67DF6,51601,77,LOCAL,BRT,GVT,PRE PAGO,300000210,Thursday,RT
2,2013-03-21,01:10:47,0.83,32,1AC8490BA99709F652B4A5C3525D3C82,32,B6BB8A38B286A41978E732BDE0249DB5,50141,117,LOCAL,OI,OI,POS PAGO,300000188,Thursday,SJDR5
3,2013-03-21,21:11:55,6.87,32,05AC98D91ED3C2821F53519B19CCBC58,32,A3ACD65D308B86F4487DB35A5F2E50EB,50141,117,LOCAL,OI,OI,PRE PAGO,300000533,Thursday,SJDR5
4,2013-03-21,20:29:21,0.13,61,BA4DEA1F710C1216F74F1255150C49D6,21,A49BA005920B8BA24BEE40F455B69B9B,51601,117,LOCAL,BRT,OI,PRE PAGO,300000210,Thursday,RT
5,2013-03-21,00:09:27,0.13,61,023CF68D76C38796F6C3B138F27AC657,21,A49BA005920B8BA24BEE40F455B69B9B,51601,117,LOCAL,BRT,OI,PRE PAGO,300000210,Thursday,RT
6,2013-03-21,22:36:30,1.22,61,B046BF714E16102B098297E89639BA30,61,A943CD0FB95D587FF5D10F61D643AD73,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210,Thursday,RT


In [12]:
presumed_residence = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/Regiao_Presumida/residence_antenna.csv', sep=';')
presumed_residence = presumed_residence.rename(columns={'user': 'USER','residence_antenna': 'ANTENNA'})
presumed_residence_with_id = pd.merge(presumed_residence, antennas_with_id[['ANTENNA', 'ANTENNA_ID']], 
                                      on='ANTENNA', how='left')
presumed_residence
presumed_residence_with_id

,USER,ANTENNA,ANTENNA_ID
0,00003F26576D1122848B6F3CEEC250E5,50141,SJDR5
1,000579E8BA1DA9E5292392473BCCCD2A,49592,LD1
2,0008962B966EF6C5571DF03F5C786677,23461,SJDR2
3,0008B7FB336065A53EC31AD06D712E28,51601,RT
4,0008B8E32E18BF4450B7CAAF0C1CE179,23441,SJDR1
...,...,...,...
59785,FFFA3339BA362F788F81DF22F3F7B6BA,27711,MDM
59786,FFFDA183352D8DD3AC2913B09BD4174F,23451,SJDR4
59787,FFFDBC5EE75F3127C262488987D6F99D,23441,SJDR1
59788,FFFE9C91DE90C5CA962FE90319E8D620,23451,SJDR4


In [15]:
cdr_merge_antennas['CALLS_LOC'] =  np.where(presumed_residence.USER.values == cdr_merge_antennas.USER_FROM.values, 
                                                     presumed_residence_with_id['ANTENNA_ID'], np.nan)

ValueError: Can only compare identically-labeled Series objects